# Movie Data Analysis - Learn Python for Data Science Portfolio Project

In this project I will be analysing a dataset with information on several movies. The project will be organised in the following fashion:

1. **Data Analysis**<br>
    Importing, cleaning and inspecting the dataset
2. **Descriptive Statistics**<br>
    Various statistics about the columns and their values
3. **Value Summaries**<br>
    Summaries and correlations beween some values
4. **Profitability Analysis**<br>
    Inspecting movie profitability with more detail
5. **Star Power**<br>
    Inspecting stars, writers, directors and correlations
6. **Conclusion**<br>
    What key insights and takeaways I have gathered from this analysis

### 1. Data analysis

Importing the libraries and then the data from the file to a dataframe:

In [1]:
import pandas as pd
import numpy as np

movie_data = pd.read_csv('movies_updated.csv')

Previewing the first 5 rows to make sure our data looks good

In [2]:
movie_data.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,"runtime,,"
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.0,Warner Bros.,"146.0,"
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.0,Columbia Pictures,"104.0,"
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.0,Lucasfilm,"124.0,"
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.0,Paramount Pictures,"88.0,"
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.0,Orion Pictures,"98.0,"


Checking number of rows and columns and looking for missing values:

In [3]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       4000 non-null   object 
 1   rating     3960 non-null   object 
 2   genre      4000 non-null   object 
 3   year       4000 non-null   int64  
 4   released   4000 non-null   object 
 5   score      4000 non-null   float64
 6   votes      4000 non-null   int64  
 7   director   4000 non-null   object 
 8   writer     3999 non-null   object 
 9   star       3999 non-null   object 
 10  country    4000 non-null   object 
 11  budget     4000 non-null   int64  
 12  gross      3831 non-null   float64
 13  company    3990 non-null   object 
 14  runtime,,  4000 non-null   object 
dtypes: float64(2), int64(3), object(10)
memory usage: 468.9+ KB


The dataset has 15 columns and 4000 rows of data and has some missing values in some columns, but not enough nor in relevant enough columns that warrants concern, so I'll leave the missing data as is.

The columns all have the type of values that they should have, except for runtime, which should be `int` and is showing as `object`. Upon inspection, this is due to the fact that all values in this column have an extra comma, and even the title of the column has two extra commas.
Let's fix that:

In [4]:
# Remove the comma from the `runtime` column
movie_data['runtime,,'] = movie_data['runtime,,'].str.replace(',', '')

# Renaming the `runtime` column
movie_data = movie_data.rename(columns={"runtime,,": "runtime"})

# Converting the `runtime` column to `int` type
movie_data['runtime'] = pd.to_numeric(movie_data['runtime'], downcast = 'integer')
    
# Printing a sample to confirm
movie_data.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.0,Warner Bros.,146
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.0,Columbia Pictures,104
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.0,Lucasfilm,124
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.0,Paramount Pictures,88
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.0,Orion Pictures,98


The data seems to be cleaner now, but it's possible to see that the `released` column has both the date of release and the country of release, so I'm going to separate that column into two, and to not generate confusion with the `country` column, where the movie was filmed, I'm renaming that column `filming_country`.

In [5]:
# Splitting the `released` column into two
movie_data["release date"] = movie_data["released"].str.extract(r'^(.*?)\s*\(')
movie_data["release country"] = movie_data["released"].str.extract(r'\((.*?)\)')

# Convert release_date to datetime type
movie_data["release date"] = pd.to_datetime(movie_data["release date"], errors="coerce")

# Drop the old column
movie_data = movie_data.drop(columns=["released"])

# Renaming the `country` column
movie_data = movie_data.rename(columns={"country": "filming country"})

# Previewing the data again
movie_data.head()

,name,rating,genre,year,score,votes,director,writer,star,filming country,budget,gross,company,runtime,release date,release country
0,The Shining,R,Drama,1980,8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.0,Warner Bros.,146,1980-06-13,United States
1,The Blue Lagoon,R,Adventure,1980,5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.0,Columbia Pictures,104,1980-07-02,United States
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.0,Lucasfilm,124,1980-06-20,United States
3,Airplane!,PG,Comedy,1980,7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.0,Paramount Pictures,88,1980-07-02,United States
4,Caddyshack,R,Comedy,1980,7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.0,Orion Pictures,98,1980-07-25,United States


### 2. Descriptive Statistics

Now that the data has been cleaned we can start addressing some statistics.

In [6]:
print('Ealiest year in our dataset:',movie_data.year.min())
print('Most recent year in our dataset:',movie_data.year.max())

Ealiest year in our dataset: 1980
Most recent year in our dataset: 2001


Because using describe on the columns `votes`, `budget` and `gross` would give results in scientific notation due to the large numbers,
I'm using separate methods instead so the results are more noticeable:

In [7]:
# From the column `votes`, checking the average number of votes, and which movies had the least and the most votes
print(f'\nAverage number of votes: {movie_data.votes.mean()}\n')
temp_var = movie_data.votes.min() # Tempp var that is used only to store values in this section
print(f'The movie(s) with the least amount of votes ({temp_var} votes):\n{movie_data[movie_data.votes == temp_var].name}\n')
temp_var = movie_data.votes.max()
print(f'The movie(s) with the most amount of votes ({temp_var} votes):\n{movie_data[movie_data.votes == temp_var].name}\n')

# From the column `Budget`, check, minimum, maximum and mean values
print(f'\nAverage budget: ${round(movie_data.budget.mean(), 2)}\n')
print(f'The standart deviation for budgets in movies is ${round(np.std(movie_data.budget), 2)}.\n')
print(f'Number of movies with budget given as \'0\':{movie_data[movie_data.budget == 0].shape[0]}\n')

temp_var = min(i for i in movie_data.budget if i > 0)
print(f'The movie(s) with the smallest budget (non-zero):\n{movie_data[movie_data.budget ==temp_var].name}\nwith a budget of ${temp_var}.\n')
temp_var = movie_data.budget.max()
print(f'The movie(s) with the biggest budget:\n{movie_data[movie_data.budget == temp_var].name}\nwith a budget of ${temp_var}.\n')

# From the column `gross`, check, minimum, maximum and mean values
print(f'\nAverage gross revenue: ${round(movie_data.gross.mean(), 2)}\n')
print(f'The standart deviation for gross revenue in movies is ${round(np.std(movie_data.gross), 2)}.\n')

temp_var = movie_data.gross.min()
print(f'The movie(s) with the smallest gross revenue:\n{movie_data[movie_data.gross ==temp_var].name}\nwith a revenue of ${temp_var}.\n')
temp_var = movie_data.gross.max()
print(f'The movie(s) with the biggest gross revenue:\n{movie_data[movie_data.gross == temp_var].name}\nwith a revenue of ${temp_var}.\n')

print('The 5 movies with the highest score are:\n')
movie_data.sort_values('score', ascending = False).head(5)


Average number of votes: 54494.5025

The movie(s) with the least amount of votes (51 votes):
472        Forever Young
841    Death of an Angel
Name: name, dtype: object

The movie(s) with the most amount of votes (2400000 votes):
2443    The Shawshank Redemption
Name: name, dtype: object


Average budget: $15427403.12

The standart deviation for budgets in movies is $22483581.91.

Number of movies with budget given as '0':1392

The movie(s) with the smallest budget (non-zero):
471    A Polish Vampire in Burbank
Name: name, dtype: object
with a budget of $3000.

The movie(s) with the biggest budget:
3045    Titanic
Name: name, dtype: object
with a budget of $200000000.


Average gross revenue: $41722023.0

The standart deviation for gross revenue in movies is $92582957.11.

The movie(s) with the smallest gross revenue:
3203    Trojan War
Name: name, dtype: object
with a revenue of $309.0.

The movie(s) with the biggest gross revenue:
3045    Titanic
Name: name, dtype: object
with a rev

,name,rating,genre,year,score,votes,director,writer,star,filming country,budget,gross,company,runtime,release date,release country
2443,The Shawshank Redemption,R,Drama,1994,9.3,2400000,Frank Darabont,Stephen King,Tim Robbins,United States,25000000,28817291.0,Castle Rock Entertainment,142,1994-10-14,United States
2444,Pulp Fiction,R,Crime,1994,8.9,1900000,Quentin Tarantino,Quentin Tarantino,John Travolta,United States,8000000,213928762.0,Miramax,154,1994-10-14,United States
2247,Schindler's List,R,Biography,1993,8.9,1200000,Steven Spielberg,Thomas Keneally,Liam Neeson,United States,22000000,322161245.0,Universal Pictures,195,1994-02-04,United States
3444,Fight Club,R,Drama,1999,8.8,1900000,David Fincher,Chuck Palahniuk,Brad Pitt,Germany,63000000,101209702.0,Fox 2000 Pictures,139,1999-10-15,United States
2445,Forrest Gump,PG-13,Drama,1994,8.8,1900000,Robert Zemeckis,Winston Groom,Tom Hanks,United States,55000000,678226133.0,Paramount Pictures,142,1994-07-06,United States


As there is a big jump from the minimum gross value to the mean and again to the maximum gross value, let's check out the percentiles:

In [8]:
print(f'First quartile of the gross revenue: {movie_data.gross.quantile(0.25)}')
print(f'Median of the gross revenue: {movie_data.gross.quantile(0.5)}')
print(f'Third quartile of the gross revenue: {movie_data.gross.quantile(0.75)}')

First quartile of the gross revenue: 2835743.0
Median of the gross revenue: 11838218.0
Third quartile of the gross revenue: 35604951.5


As seen above, the gross revenue goes from \\$309 to \\$2835743 in the first quartile, then to \\$11838218, quadrupling its value for the second quartile, then tripling its value to \\$35604951.5 in the third quartile, and then to a maximum of \\$2201647264, almost 62 times the gross revenue in the third quartile!

This is possibly due to some outliers, so let's have a look at which are the top-grossing movies in our dataset:

In [9]:
movie_data.sort_values('gross', ascending = False).head(10)

,name,rating,genre,year,score,votes,director,writer,star,filming country,budget,gross,company,runtime,release date,release country
3045,Titanic,PG-13,Drama,1997,7.8,1100000,James Cameron,James Cameron,Leonardo DiCaprio,United States,200000000,2.201647e+09,Twentieth Century Fox,194,1997-12-19,United States
2448,The Lion King,G,Animation,1994,8.5,970000,Roger Allers,Irene Mecchi,Matthew Broderick,United States,45000000,1.083721e+09,Walt Disney Pictures,88,1994-06-24,United States
2245,Jurassic Park,PG-13,Action,1993,8.1,894000,Steven Spielberg,Michael Crichton,Sam Neill,United States,63000000,1.033928e+09,Universal Pictures,127,1993-06-11,United States
3451,Star Wars: Episode I - The Phantom Menace,PG,Action,1999,6.5,748000,George Lucas,George Lucas,Ewan McGregor,United States,115000000,1.027083e+09,Lucasfilm,136,1999-05-19,United States
3843,Harry Potter and the Sorcerer's Stone,PG,Adventure,2001,7.6,685000,Chris Columbus,J.K. Rowling,Daniel Radcliffe,United Kingdom,125000000,1.006968e+09,Warner Bros.,152,2001-11-16,United States
3845,The Lord of the Rings: The Fellowship of the Ring,PG-13,Action,2001,8.8,1700000,Peter Jackson,J.R.R. Tolkien,Elijah Wood,New Zealand,93000000,8.976901e+08,New Line Cinema,178,2001-12-19,United States
2844,Independence Day,PG-13,Action,1996,7.0,543000,Roland Emmerich,Dean Devlin,Will Smith,United States,75000000,8.174009e+08,Twentieth Century Fox,145,1996-07-03,United States
209,E.T. the Extra-Terrestrial,PG,Family,1982,7.8,381000,Steven Spielberg,Melissa Mathison,Henry Thomas,United States,10500000,7.929106e+08,Universal Pictures,115,1982-06-11,United States
2445,Forrest Gump,PG-13,Drama,1994,8.8,1900000,Robert Zemeckis,Winston Groom,Tom Hanks,United States,55000000,6.782261e+08,Paramount Pictures,142,1994-07-06,United States
3453,The Sixth Sense,PG-13,Drama,1999,8.1,930000,M. Night Shyamalan,M. Night Shyamalan,Bruce Willis,United States,40000000,6.728063e+08,Hollywood Pictures,107,1999-08-06,United States


As the `runtime` column has smaller numbers, we can get the summary statistics using the `describe()` method:

In [10]:
movie_data.runtime.describe()

count    4000.000000
mean      105.711750
std        18.470556
min        55.000000
25%        94.000000
50%       102.000000
75%       114.000000
max       357.000000
Name: runtime, dtype: float64

The shortest film has a runtime of only 55 minutes, while the longest has a runtime of almost 6 hours! Most movies have a length of about 1 hour and 45 minutes.
The movies with the shortest and the longest runtimes are:

In [11]:
print('Shortest runtime:', movie_data[movie_data.runtime == 55].name, '\n')
print('Longest runtime:', movie_data[movie_data.runtime == 357].name)

Shortest runtime: 474    The Business of Show Business
Name: name, dtype: object 

Longest runtime: 1218    Little Dorrit
Name: name, dtype: object


In [12]:
print('Dates with most released movies:')
print(movie_data['release date'].mode())
print(f'Those dates had a total of {movie_data[movie_data['release date'] == '1991-10-04'].shape[0]} movies released each.')


Dates with most released movies:
0   1986-02-14
1   1991-10-04
Name: release date, dtype: datetime64[ns]
Those dates had a total of 9 movies released each.


Looking at the dates above, I got curious and decided to investigate if, accross the years, there's a peculiar day/month date that has more releases than the rest. For that, I'm going to format the `release date` column to take out the year (as the year is already present in the `year` column) and rerun the code above:

In [30]:
movie_data['release date'] = movie_data['release date'].dt.strftime('%m-%d')

print('Dates with most released movies:')
best_release_date = movie_data['release date'].mode().iloc[0] # As mode returns a series, using iloc makes it only returning the first value
print(best_release_date)
print(f'That date had a total of {movie_data[movie_data['release date'] == best_release_date].shape[0]} movies released.')

Dates with most released movies:
12-25
That date had a total of 49 movies released.


Now that we analysed the numerical columns, let's have a look at some categorical columns:

In [14]:
print(movie_data.genre.describe(),'\n')
print(movie_data.company.describe(),'\n')
print(movie_data.director.describe(),'\n')
print(movie_data.star.describe(),'\n')
print(movie_data.writer.describe(),'\n')
print(movie_data.rating.describe())

count       4000
unique        17
top       Comedy
freq        1355
Name: genre, dtype: object 

count                   3990
unique                  1305
top       Universal Pictures
freq                     191
Name: company, dtype: object 

count            4000
unique           1678
top       Woody Allen
freq               22
Name: director, dtype: object 

count               3999
unique              1655
top       Robert De Niro
freq                  27
Name: star, dtype: object 

count             3999
unique            2685
top       Stephen King
freq                27
Name: writer, dtype: object 

count     3960
unique      11
top          R
freq      2043
Name: rating, dtype: object


Here, we can see that the most common movie genre is 'comedy' with 1355 occurrences, with over a quarter of the movies in our dataset. The most common rating is also 'R' - 'Restricted' with over half the movies obtaining that rating.

### 3. Value Summaries

Let's now get some more details in some values. I'll start by analysing how many counts of each rating there are:

In [15]:
movie_data.rating.value_counts()

rating
R            2043
PG-13         848
PG            815
G             106
Not Rated      92
Unrated        32
NC-17          14
TV-MA           4
X               3
TV-PG           2
Approved        1
Name: count, dtype: int64

As we've seen, 'R' rated movies are the majority, with over twice the second or third places, 'PG-13' and 'PG' which have 848 and 815 counts each, followed by a big fall for 'G' rated movies with only 106 counts.

Let's do the same for the `genre`:

In [16]:
movie_data.genre.value_counts()

genre
Comedy       1355
Drama         807
Action        775
Crime         302
Adventure     231
Biography     175
Horror        158
Animation     113
Fantasy        33
Mystery        13
Thriller       11
Family          8
Romance         8
Sci-Fi          6
Western         3
Music           1
History         1
Name: count, dtype: int64

Right after 'Comedy', the next two most popular genres are 'Drama' and 'Action' with around 800 counts each, followed by 'Crime' with only 302 movies released in this genre.

I'll now try to find out if the runtime of the films changed as the years went by:

In [17]:
print(movie_data.groupby('year').agg({'runtime': 'mean'}).reset_index())

    year     runtime
0   1980  107.402174
1   1981  105.955752
2   1982  105.928571
3   1983  102.270833
4   1984  104.446429
5   1985  102.235000
6   1986  101.805000
7   1987  102.710000
8   1988  103.585000
9   1989  102.710000
10  1990  104.925000
11  1991  106.315000
12  1992  106.690000
13  1993  107.525000
14  1994  105.570000
15  1995  106.725000
16  1996  107.240000
17  1997  107.260000
18  1998  107.365000
19  1999  109.390000
20  2000  107.850000
21  2001  110.777070


We can see that despite a little dip in the 80s, the average runtime of the movies has been very consistent, always in close proximity to 105 minutes.
Let's do the same for `score`, `budget` and `gross`:

In [18]:
movie_data.groupby('year').agg({'score': 'mean','budget': 'mean', 'gross': 'mean'}).round(2).reset_index()

,year,score,budget,gross
0,1980,6.30,6183152.17,30662555.11
1,1981,6.30,6278318.58,24231393.16
2,1982,6.29,6640238.10,26786390.90
3,1983,6.02,6208163.62,21484411.14
4,1984,6.19,6644797.62,22361343.01
5,1985,6.31,6102550.00,20855984.65
6,1986,6.15,6323936.88,19610278.69
7,1987,6.22,6912713.39,21454437.53
8,1988,6.28,8652700.00,25342333.15
9,1989,6.18,8923812.89,32207676.08


Through these observations we can see that the average score has remained fairly consistent through the years, but the average budget for the movies has increased almost 6 times, while their gross revenue increased by only three times. This is a bit surprising, as my expectation was for the gross revenue to be directly correlated to the budget. The fact that the budget still keeps increasing but the gross revenue is not increasing as much might be because the production companies are also counting on other types of revenue to make up for the budget allocated to filming, like merchandise, books, games, DVD rentals, TV airings, etc.

Now, let's investigate the 'rating' and 'genre' columns:

In [19]:
movie_data.groupby('rating').agg({'score': 'mean','budget': 'mean', 'gross': 'mean', 'runtime': 'mean'}).round(2).reset_index()

,rating,score,budget,gross,runtime
0,Approved,3.40,6500000.00,36565280.00,115.00
1,G,6.70,21948113.21,88502424.67,89.60
2,NC-17,6.53,4132142.86,4820668.21,102.57
3,Not Rated,7.14,502532.61,2302852.06,109.97
4,PG,6.17,14752176.40,45093268.10,103.82
5,PG-13,6.19,23811162.46,65933528.33,106.12
6,R,6.36,13172908.77,30495336.76,106.89
7,TV-MA,7.22,0.00,2273019.25,102.25
8,TV-PG,5.85,0.00,1689723.00,106.50
9,Unrated,6.79,655117.78,2081509.04,108.75


'TV-MA' ratings have the best average score, while 'TV-PG' have the lowest, as 'Approved' is not an official rating. The most budget is invested in 'PG-13' and 'G' ratings, which are the ones that generate the highest amounts of gross revenue. As for runtime, as the rating goes higher in the hierarchy, the longer the average runtime.

In [20]:
movie_data.groupby('genre').agg({'score': 'mean','budget': 'mean', 'gross': 'mean', 'runtime': 'mean'}).round(2).reset_index()

,genre,score,budget,gross,runtime
0,Action,6.01,26622223.17,7.232715e+07,105.94
1,Adventure,6.15,19068658.01,4.786276e+07,106.18
2,Animation,6.88,29784955.75,1.004415e+08,86.62
3,Biography,7.05,14157312.96,3.012749e+07,122.43
4,Comedy,6.18,11095983.18,3.108453e+07,100.49
5,Crime,6.65,14407662.25,2.953531e+07,111.48
6,Drama,6.67,11349910.76,3.089225e+07,113.41
7,Family,6.55,3687500.00,1.098218e+08,97.00
8,Fantasy,6.02,16657575.76,3.583649e+07,99.09
9,History,8.30,323562.00,NaN,55.00


As for the genres of the films, the highest-rated genre is 'History', followed by 'Music' and then 'Biography', and the lowest scores belong to 'Western', followed by 'Horror' and 'Thriller', but these results might be due to the reduced number of movies from these 6 categories, skewing the data.
The biggest budgets are allocated to 'Action' and 'Animated' movies, which coincide with the movies which return the most gross revenue. 'Music' and 'Biography' genres have noticeably longer runtimes than the other categories, and the 'History' genre has the lowest runtime at 55 minutes.

### 4. Profitability Analysis

To better analyse the profitability of the movie industry, I'll create a new column in the dataset that contains a 'profit ratio'
$$profit\_ratio = \frac{gross\_revenue}{budget}$$

In [21]:
movie_data['profit ratio'] = (movie_data.gross/movie_data.budget).round(2)
movie_data.head(5)

,name,rating,genre,year,score,votes,director,writer,star,filming country,budget,gross,company,runtime,release date,release country,profit ratio
0,The Shining,R,Drama,1980,8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.0,Warner Bros.,146,1980-06-13,United States,2.47
1,The Blue Lagoon,R,Adventure,1980,5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.0,Columbia Pictures,104,1980-07-02,United States,13.08
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.0,Lucasfilm,124,1980-06-20,United States,29.91
3,Airplane!,PG,Comedy,1980,7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.0,Paramount Pictures,88,1980-07-02,United States,23.84
4,Caddyshack,R,Comedy,1980,7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.0,Orion Pictures,98,1980-07-25,United States,6.64


Now we can find out what genres, ratings and companies were the most profitable:

In [22]:
print(movie_data.groupby('genre').agg({'profit ratio': 'mean'}).round(2).reset_index(), '\n')
print(movie_data.groupby('rating').agg({'profit ratio': 'mean'}).round(2).reset_index(), '\n')
print(movie_data.groupby('company').agg({'profit ratio': 'mean'}).round(2).reset_index())

        genre  profit ratio
0      Action           inf
1   Adventure           inf
2   Animation           inf
3   Biography           inf
4      Comedy           inf
5       Crime           inf
6       Drama           inf
7      Family           inf
8     Fantasy           inf
9     History           NaN
10     Horror           inf
11      Music           inf
12    Mystery           inf
13    Romance           inf
14     Sci-Fi           inf
15   Thriller           inf
16    Western           inf 

       rating  profit ratio
0    Approved          5.63
1           G           inf
2       NC-17           inf
3   Not Rated           inf
4          PG           inf
5       PG-13           inf
6           R           inf
7       TV-MA           inf
8       TV-PG           inf
9     Unrated           inf
10          X           inf 

                          company  profit ratio
0                 .406 Production          0.04
1              1+2 Seisaku Iinkai           inf
2           

As can be seen above, there are a lot of infinite values, implying that in our dataset we have a lot of '0' values. I'm going to see how many values of the column `budget` are '0':

In [23]:
movie_data.budget.value_counts().head()

budget
0           1392
10000000     114
15000000     113
25000000     100
20000000      93
Name: count, dtype: int64

Our data does contain a lot of values for `budget` that are zeros, so I'm going to investigate the same as above, but only for `budget` greater than '0'.<br> Also, as there are many different companies in our dataset, let's order the set so we can have a look at the top 3 companies:

In [24]:
print(movie_data.loc[movie_data['budget']>0].groupby('genre').agg({'profit ratio': 'mean'}), '\n')
print(movie_data.loc[movie_data['budget']>0].groupby('rating').agg({'profit ratio': 'mean'}).round(2).reset_index(), '\n')
print(movie_data.loc[movie_data['budget']>0].groupby('company').agg({'profit ratio': 'mean'}).round(2).sort_values('profit ratio', ascending = False).reset_index().head(3))

           profit ratio
genre                  
Action         3.185255
Adventure      2.338944
Animation      3.630000
Biography      2.248835
Comedy         3.585496
Crime          1.928065
Drama          2.744063
Family        37.815000
Fantasy        2.319355
History             NaN
Horror        40.836930
Mystery        2.430000
Romance        0.616667
Sci-Fi         1.370000
Thriller       1.533333
Western        0.995000 

      rating  profit ratio
0   Approved          5.63
1          G          3.59
2      NC-17          2.80
3  Not Rated          2.10
4         PG          2.98
5      PG-13          2.63
6          R          6.41
7    Unrated          5.05
8          X          1.01 

                  company  profit ratio
0             Haxan Films       4143.98
1           Redwave Films         73.70
2  View Askew Productions         59.92


Now, we can see that for budgets reported greater than '0', the 'Family' and 'Horror' genres have a profit ratio way above all other genres, with approximately a 40% ratio, while all other genres are within the 1 to 3.5% ratio. 'R'-rated movies tend to have a better ratio as well, at 6.41%, while 'X'-rated movies have the worst ratio at 1.01%. The company with the best ratio is Haxan Films, with a ratio of 4143.93, way above their spent budget!

### 5. Star Power

I'm now going to dig deeper into analysis and correlations derived from the actors, directors and writers. Let's start by observing the directors:

In [35]:
print('The top five director/actor colaborations are:\n',
      movie_data.groupby(['director','star']).size().reset_index(name="count").sort_values("count", ascending=False).head(5), '\n')

print('The top five director/writer colaborations are:\n',
      movie_data.groupby(['director', 'writer']).size().reset_index(name="count").sort_values("count", ascending=False).head(5), '\n')

print('The top three director/company colaborations are:\n',
      movie_data.groupby(['director', 'company']).size().reset_index(name="count").sort_values("count", ascending=False).head(3), '\n')

The top five director/actor colaborations are:
              director             star  count
589    Clint Eastwood   Clint Eastwood     13
3697      Woody Allen      Woody Allen     12
1326  J. Lee Thompson  Charles Bronson      6
2840   Richard Donner       Mel Gibson      6
0        Aaron Norris     Chuck Norris      5 

The top five director/writer colaborations are:
              director           writer  count
3517      Woody Allen      Woody Allen     21
3074        Spike Lee        Spike Lee      8
305     Blake Edwards    Blake Edwards      8
1624      John Hughes      John Hughes      8
2485  Pedro Almodóvar  Pedro Almodóvar      8 

The top three director/company colaborations are:
             director                                      company  count
3532     Woody Allen  Jack Rollins & Charles H. Joffe Productions     11
571   Clint Eastwood                                 Warner Bros.      8
1334     James Ivory                   Merchant Ivory Productions      7 



Let's see who the top 10 directors with best score are:

In [26]:
movie_data.groupby('director').agg({'score': 'mean'}).reset_index().round(1).sort_values('score', ascending = False).head(10)

,director,score
1601,Tony Kaye,8.5
1378,Roger Allers,8.5
1437,Sergio Leone,8.4
1583,Tom Logan,8.3
1534,Tengiz Abuladze,8.3
1136,Moustapha Akkad,8.3
1416,Sam Mendes,8.3
595,Hayao Miyazaki,8.3
470,Frank Darabont,8.3
200,Béla Tarr,8.2


From the above information, we can see that both Clint Eastwood and Woody Allen have the most movies as both director and star, as they have 13 and 12 movies where they are the main actor. Woody Allen also wrote 21 movies in which he was also the director. Both Woody Allen and Clint Eastwood appear again in the top three for director/company collaborations with Jack Rollins & Charles H. Joffe Productions and Warner Bros., respectively.<br>
The top leading directors are more varied, with both Tony Kaye and Roger Allers in first place with a score of 8.5.


I'm now going to check which actors work most often with which companies and which genres and then see a list of the 10 best-scored actors:

In [27]:
print('The top three actor/company colaborations are:\n',
      movie_data.groupby(['star','company']).size().reset_index(name="count").sort_values("count", ascending=False).head(3), '\n')

print('The top three actor on genre works are:\n',
      movie_data.groupby(['star','genre']).size().reset_index(name="count").sort_values("count", ascending=False).head(3), '\n')

movie_data.groupby('star').agg({'score': 'mean'}).reset_index().round(1).sort_values('score', ascending = False).head(10)

The top three actor/company colaborations are:
                 star             company  count
608   Clint Eastwood        Warner Bros.      8
902     Eddie Murphy  Paramount Pictures      7
3204   Steven Seagal        Warner Bros.      7 

The top three actor on genre works are:
                        star   genre  count
2253           Steve Martin  Comedy     20
129   Arnold Schwarzenegger  Action     18
411            Chuck Norris  Action     17 



,star,score
321,Daveigh Chase,8.6
1273,Philippe Noiret,8.5
411,Edward Norton,8.5
1033,Mark Hamill,8.5
1649,Yôji Matsuda,8.4
456,F. Murray Abraham,8.3
93,Audrey Tautou,8.3
80,Anthony Quinn,8.3
94,Avtandil Makharadze,8.3
585,Hossain Sabzian,8.3


Clint Eastwood appears again, leading the collaboration between a star and company, Warner Bros, with 8 movies, and the top 3 actors working in a genre are lead by Steve Martin in "Comedy" with 20 movies, followed closely by Arnold Schwarzenegger and Chuck Norris in "Action" with 18 and 17.<br>
Stars with higher scores are Daveigh Chase with 8.6 and, with 8.5, actors Philippe Noiret, Edward Norton and Mark Hamill.

We can also have a quick look at which writers had the movies with higher scores:

In [28]:
movie_data.groupby('writer').agg({'score': 'mean'}).reset_index().round(1).sort_values('score', ascending = False).head(10)

,writer,score
2468,Thomas Keneally,8.9
1031,J.R.R. Tolkien,8.8
422,Chuck Palahniuk,8.8
2666,Winston Groom,8.8
1504,Leigh Brackett,8.7
2580,Vincenzo Cerami,8.6
1003,Irene Mecchi,8.5
940,Harry Grey,8.4
1843,Nicholas Pileggi,8.4
1975,Peter Shaffer,8.3


From this data it's possible to see that the majority of the names that appear are novelists and writers whose works have been adapted to the big screen, including the leading 5 names.

### 6. Conclusion

I had a lot of fun and made some interesting discoveries while doing this project, and it got me curious to know, if it was a dataset with more recent entries, how would the findings change? I would definitely like to work with a larger and more recent dataset on movies one day, should the opportunity arise.

These are some findings I made during the analysis:
 - Titanic is the movie with the biggest budget and gross revenue in our dataset, having a budget of \\$200M and a gross revenue of \\$2B.
 - Of the top 5 best-scored movies from 1980 to 2001, 4 are from 1994: "The Shawshank Redemption", "Pulp Fiction", and "Schindler's List" in the first three positions and "Forrest Gump" in number 5.
 - When analysing the gross revenue, there is a big jump from the third quartile to the maximum values, with the maximum being almost 62 times the value of the third quartile. This due to movies that managed to have gross revenue several times their budget, like "Titanic", "The Lion King" or "Jurassic Park".
 - The dates with the most simultaneous releases were 14/02/86, Valentine's Day, and 04/10/1991, both with 9 movies released that day, but the best day over all the years to release movies is Christmas Day, with 49 movies released on Christmas, possibly due to people having Christmas breaks and holidays.
 - The most frequent genre is Comedy, with 1355 entries, followed by Drama with 807 and Action with 775. The company with the most releases is Universal Pictures with 191 movies, and "R"-rated movies are more frequent with 2043 entries, over half of our dataset!
 - Over the years, the average runtimes and score have remained fairly consistent.
 - The movie's production budget has increased almost 6 times, while the gross revenue has only increased approximately 3 times, which I found interesting, as I was expecting the gross revenue to have increased more or the same as the budget. This might be because the production companies might be relying more on other sources of revenue like toys, TV airings, memorabilia, etc.
 - Both the biggest budget and gross revenue are for "PG-13" and "PG" ratings, which makes sense, as everyone of every age can see them, and this makes a wider audience.
 - In genres, "Animation" and "Action" movies are the ones with higher budgets and also higher gross revenue.
 - If we take into account the profitability ratio, "Family" and "Horror" movies are the genres with the best ratio, of around 40% gross revenue for their budget, while all other genres remain between 1 and 3.5%. "R"-rated movies also have the best profitability ratio at 6.41%.
 - Both Clint Eastwood and Woody Allen starred in movies they were directing 13 and 12 times, respectively, but other than that, the most collaborations between movie directors and actors go to J. Lee Thompson with actor Charles Bronson and Director Richard Dormer with Mel Gibson, with 6 movies each.<br> Woody also wrote 21 out of the 22 movies he directed, being the director with the most movies released.
 - Woody Allen and Clint Eastwood appear again leading the top of directors working with the same production companies, with Allen collaborating with Jack Rollins & Charles H. Joffe Productions 11 times and Eastwood releasing 8 films with Warner Bros. <br> Eastwood is also the main actor in movies from Warner Bros., being the main actor in the 8 movies he directed for that company.
 - There is a clear tendency for directors to be the writers of the movies they are directing, but the writer with the most movies released is Stephen King with 27 movie adaptations.
 - The most highly scored writers are novelists and writers whose work has been adapted for the big screen, like Thomas Keneally and J.R.R. Tokien.